In [20]:
import pandas as pd
import numpy as np
import pandahouse as ph
from datetime import date
import holidays

In [21]:
connection = dict(database='test',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

**6. Давайте посмотрим на топ-10 платежей по сумме в рублях. Выведите первые 10 строк таблицы checks, но теперь отсортировав значения Rub по убыванию (DESC).**

In [23]:
query = """
SELECT 
    *
FROM default.checks
order by Rub DESC
LIMIT 10
"""
df = ph.read_clickhouse(query, connection=connection)
df

,Rub,BuyDate,UserID
0,181063,2019-06-04,15605251414578189718
1,66318,2019-09-16,9613023063681218234
2,56327,2019-01-18,10398072986532593332
3,48758,2019-08-19,16967914029594691558
4,42945,2019-09-13,3216678490746804369
5,38989,2019-05-31,287271529430059152
6,38555,2019-04-03,6890024450198999904
7,38189,2019-03-22,17121049924398614106
8,35769,2019-02-05,9002023542637148786
9,34396,2019-02-26,5050480259433864828


**7. Для работы бывает удобно переименовать колонки таблицы. Выведите первые 15 строк таблицы checks, переименовав поле Rub в Revenue и используя сортировку по возрастанию столбца UserID. Разместите столбцы в запросе в следующем порядке revenue, buydate, userid.**

In [25]:
query = """
SELECT 
    Rub as Revenue,
    BuyDate,
    UserID
FROM default.checks
order by UserID
LIMIT 15
"""
df = ph.read_clickhouse(query, connection=connection)
df

,Revenue,BuyDate,UserID
0,3,2019-07-29,1406814346437
1,4,2019-07-27,1406814346437
2,5,2019-03-10,3116519148596
3,0,2019-09-14,3827093218449
4,0,2019-09-01,3827093218449
5,21,2019-08-31,3827093218449
6,1,2019-06-06,3827093218449
7,9,2019-01-11,4301734536835
8,11,2019-01-15,4301734536835
9,0,2019-05-29,9820728992879


**8. Проверьте, данные за какой период имеются в таблице. Для этого используйте MIN и MAX по колонке BuyDate. Колонки с минимальным и максимальным значением назовите MinDate и MaxDate соответственно.**

In [29]:
query = """
SELECT 
    min(BuyDate) as MinDate,
    max(BuyDate) as MaxDate
FROM default.checks
LIMIT 15
"""
df = ph.read_clickhouse(query, connection=connection)
df

,MinDate,MaxDate
0,2019-01-01,2019-10-04


**9. Предположим, что перед нами стоит задача отобрать несколько групп пользователей по разным критериям, чтобы отправить им персональные сообщения об акциях. Например, первым 10 пользователям, зарегистрировавшимся на сайте (чем раньше зарегистрировался пользователь, тем меньше его UserID), мы хотим подарить скидку на следующую покупку.**

**Выведите UserID 10 уникальных покупателей, используя оператор DISTINCT. Таблицу отсортируйте по UserID в порядке возрастания.**


In [30]:
query = """
SELECT 
    distinct UserID
FROM default.checks
order by UserID
LIMIT 10
"""
df = ph.read_clickhouse(query, connection=connection)
df

,UserID
0,1406814346437
1,3116519148596
2,3827093218449
3,4301734536835
4,9820728992879
5,10995267265098
6,13652596829677
7,20964545223437
8,20990265325703
9,21272350883382


**10. Мы хотим сформировать новую акцию к 8 марта, и для этого нас интересуют самые крупные покупки, совершенные в этот день в прошлом году.**

**Выведите информацию о продажах, которые были совершены 8 марта 2019 года, используя блок WHERE применительно к полю BuyDate. Значения отсортируйте по убыванию Rub. Количество строк для извлечения – 10. Дата хранится в таблице в текстовом виде и имеет формат год-месяц-день (‘YYYY-MM-DD’). Например: 1 декабря 2018 года записывается как '2018-12-01'.**


In [33]:
query = """
SELECT 
    *
FROM default.checks
WHERE BuyDate = '2019-03-08'
order by Rub DESC
LIMIT 10
"""
df = ph.read_clickhouse(query, connection=connection)
df

,Rub,BuyDate,UserID
0,1864,2019-03-08,16554229243029267842
1,1678,2019-03-08,12413854814605490528
2,1492,2019-03-08,18030280579764769170
3,1384,2019-03-08,10707840839029652796
4,1377,2019-03-08,12440720654593005906
5,1259,2019-03-08,18438892581701323582
6,1251,2019-03-08,2500270806192827116
7,1205,2019-03-08,11137756530252864327
8,1182,2019-03-08,1521860760741361000
9,1166,2019-03-08,369532348365642666



**11. Мы предполагаем, что пользователи, которые совершили крупные покупки 1 сентября в прошлом году - родители школьников, и в следующем году можно будет подготовить для них акцию на группу товаров.**

**Давайте применим WHERE к полям BuyDate и Rub, и посмотрим на уникальные id покупателей из checks, которые совершили покупку 1 сентября 2019 года на сумму более 2 000 рублей. Значения отсортируйте по убыванию UserID.**

In [42]:
query = """
SELECT 
    Distinct UserID
FROM default.checks
WHERE Rub > 2000 and BuyDate = '2019-09-01'
order by UserID desc
LIMIT 10
"""
df = ph.read_clickhouse(query, connection=connection)
df

,UserID
0,14038271144877258618
1,75940702290031146
